In [ ]:
!pip install confluent-kafka cassandra-driver

In [3]:
from confluent_kafka import Consumer, KafkaError
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement

# Kafka Consumer configuration
conf = {
    'bootstrap.servers': 'localhost:9092',  # Replace with your Kafka broker's address
    'group.id': '_confluent-controlcenter-5-4-0-1-command',       # Consumer group ID
    'auto.offset.reset': 'earliest'       # Start consuming from the beginning of the topic
}

# Create the Kafka Consumer
consumer = Consumer(conf)

# Subscribe to the Kafka topic
topic = 'my_topic-1'  # Replace with the topic you want to consume from
consumer.subscribe([topic])

# Cassandra configuration
cluster = Cluster(['localhost'])  # Replace with your Cassandra cluster's contact points
session = cluster.connect()

key = session.execute('DESCRIBE KEYSPACES')
print("Cassandra Keyspaces:- ",key)

session.execute('use kafkalearning')

# Cassandra table schema (if not already created)
create_table_query = '''
    CREATE TABLE IF NOT EXISTS kafka_cassandra_table ( id int,prod_name text,expiry_date date, rate int , created_at TIMESTAMP, updated_at  TIMESTAMP, PRIMARY KEY(id) )
'''
session.execute(create_table_query)

# Consume and process messages
try:
    while True:
        msg = consumer.poll(1.0)

        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                print(f"Reached end of partition: {msg.topic()} [{msg.partition()}]")
            else:
                print(f"Error while consuming from topic {msg.topic()}: {msg.error()}")
        else:
            # Process the message and insert into Cassandra
            id, prod_name,expiry_date,rate,created_at,updated_at = msg.value().decode('utf-8').split(':')
            insert_query = '''
                INSERT INTO my_cassandra_table (id, prod_name,expiry_date,rate,created_at,updated_at) VALUES (%s, %s,%s,%s,%s,%s)
            '''
            session.execute(insert_query, (int(id), prod_name,expiry_date,int(rate),created_at,updated_at))

except KeyboardInterrupt:
    pass

finally:
    # Close the Kafka Consumer and Cassandra session
    consumer.close()
    session.shutdown()
    cluster.shutdown()


Cassandra Keyspaces:-  <cassandra.cluster.ResultSet object at 0x00000223F457D690>
Error while consuming from topic None: KafkaError{code=INCONSISTENT_GROUP_PROTOCOL,val=23,str="JoinGroup failed: Broker: Inconsistent group protocol"}
Error while consuming from topic None: KafkaError{code=INCONSISTENT_GROUP_PROTOCOL,val=23,str="JoinGroup failed: Broker: Inconsistent group protocol"}
Error while consuming from topic None: KafkaError{code=INCONSISTENT_GROUP_PROTOCOL,val=23,str="JoinGroup failed: Broker: Inconsistent group protocol"}
Error while consuming from topic None: KafkaError{code=INCONSISTENT_GROUP_PROTOCOL,val=23,str="JoinGroup failed: Broker: Inconsistent group protocol"}
Error while consuming from topic None: KafkaError{code=INCONSISTENT_GROUP_PROTOCOL,val=23,str="JoinGroup failed: Broker: Inconsistent group protocol"}
Error while consuming from topic None: KafkaError{code=INCONSISTENT_GROUP_PROTOCOL,val=23,str="JoinGroup failed: Broker: Inconsistent group protocol"}
Error while 